In [9]:
# Импорт библиотек
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.cluster import KMeans
from scipy.stats import mode

**Теория:**

Базовый алгоритм совместной фильтрации.

$$\hat{r}_{ui} = \frac{ \sum\limits_{v \in N^k_i(u)} \text{sim}(u, v) \cdot r_{vi}} {\sum\limits_{v \in N^k_i(u)} \text{sim}(u, v)}$$
или
$$\hat{r}_{ui} = \frac{ \sum\limits_{j \in N^k_u(i)} \text{sim}(i, j) \cdot r_{uj}} {\sum\limits_{j \in N^k_u(i)} \text{sim}(i, j)}$$

Однако, в работе нужно использовать базовый алгоритм совместной фильтрации, учитывающий средние оценки каждого пользователя.

$$
\hat{r}_{ui} = \mu_u + \frac{ \sum\limits_{v \in N^k_i(u)} \text{sim}(u, v) \cdot (r_{vi} - \mu_v)} {\sum\limits_{v \in N^k_i(u)} \text{sim}(u, v)}
$$
или
$$
\hat{r}_{ui} = \mu_i + \frac{ \sum\limits_{j \in N^k_u(i)} \text{sim}(i, j) \cdot (r_{uj} - \mu_j)} {\sum\limits_{j \in N^k_u(i)} \text{sim}(i, j)}
$$


Но в итоге можно задать `use_mean_adjustment = (False/True)` для выбора алгоритма.

In [10]:
# 1. Загрузка и разделение данных
def load_and_split_data(file_path, test_size=0.3, random_state=42):
    df = pd.read_csv(file_path)
    train, test = train_test_split(df, test_size=test_size, random_state=random_state)
    return train, test

# 2. User-based коллаборативная фильтрация
def user_based_cf(train, test, use_mean_adjustment=False):
    # Создаем матрицу пользователь-фильм
    user_item_matrix = train.pivot(index='userId', columns='movieId', values='rating').fillna(0)
    
    # Вычисляем косинусное сходство между пользователями
    user_similarity = cosine_similarity(user_item_matrix)
    
    # Вычисляем средний рейтинг для каждого пользователя (если нужно)
    user_means = user_item_matrix.mean(axis=1) if use_mean_adjustment else None
    
    # Функция для предсказания рейтинга
    def predict_rating(user_id, item_id):
        # Если юзер или фильм отсутствуют в базе, то выдаем среднее значение
        if user_id not in user_item_matrix.index or item_id not in user_item_matrix.columns:
            return user_item_matrix.mean().mean()
        
        user_ratings = user_item_matrix.loc[user_id]
        similar_users = user_similarity[user_item_matrix.index.get_loc(user_id)]
        
        # Находим топ-10 похожих пользователей
        top_similar_users = np.argsort(similar_users)[-40:-1]
        
        # Вычисляем взвешенный рейтинг
        numerator = 0
        denominator = 0
        for similar_user in top_similar_users:
            if user_item_matrix.iloc[similar_user][item_id] > 0:
                # Если выбран алгоритм с учетом средних оценок
                if use_mean_adjustment:
                    adjusted_rating = user_item_matrix.iloc[similar_user][item_id] - user_means.iloc[similar_user]
                    numerator += similar_users[similar_user] * adjusted_rating
                else:
                    numerator += similar_users[similar_user] * user_item_matrix.iloc[similar_user][item_id]
                
                denominator += similar_users[similar_user]
        
        if denominator == 0:
            return user_ratings.mean()
        
        # Если выбран алгоритм с учетом средних оценок
        if use_mean_adjustment:
            return user_means.loc[user_id] + numerator / denominator
        else:
            return numerator / denominator
    
    # Предсказываем рейтинги для тестового набора
    test['predicted_rating'] = test.apply(lambda row: predict_rating(row['userId'], row['movieId']), axis=1)
    
    # Вычисляем RMSE
    rmse = np.sqrt(mean_squared_error(test['rating'], test['predicted_rating']))
    return rmse

# 3. Item-based коллаборативная фильтрация
def item_based_cf(train, test, use_mean_adjustment=False):
    # Создаем матрицу фильм-пользователь
    item_user_matrix = train.pivot(index='movieId', columns='userId', values='rating').fillna(0)
    
    # Вычисляем косинусное сходство между фильмами
    item_similarity = cosine_similarity(item_user_matrix)
    
    # Вычисляем средний рейтинг для каждого фильма (если нужно)
    item_means = item_user_matrix.mean(axis=1) if use_mean_adjustment else None
    
    # Функция для предсказания рейтинга
    def predict_rating(user_id, item_id):
        if item_id not in item_user_matrix.index or user_id not in item_user_matrix.columns:
            return item_user_matrix.mean().mean()
        
        item_ratings = item_user_matrix.loc[item_id]
        similar_items = item_similarity[item_user_matrix.index.get_loc(item_id)]
        
        # Находим топ-10 похожих фильмов
        top_similar_items = np.argsort(similar_items)[-40:-1]
        
        # Вычисляем взвешенный рейтинг
        numerator = 0
        denominator = 0
        for similar_item in top_similar_items:
            if item_user_matrix.iloc[similar_item][user_id] > 0:
                if use_mean_adjustment:
                    # Коррекция рейтинга на среднее значение
                    adjusted_rating = item_user_matrix.iloc[similar_item][user_id] - item_means.iloc[similar_item]
                    numerator += similar_items[similar_item] * adjusted_rating
                else:
                    numerator += similar_items[similar_item] * item_user_matrix.iloc[similar_item][user_id]
                denominator += similar_items[similar_item]
        
        if denominator == 0:
            return item_ratings.mean()
        
        if use_mean_adjustment:
            # Предсказание с корректировкой на среднее значение фильма
            return item_means.loc[item_id] + numerator / denominator
        else:
            return numerator / denominator
    
    # Предсказываем рейтинги для тестового набора
    test['predicted_rating'] = test.apply(lambda row: predict_rating(row['userId'], row['movieId']), axis=1)
    
    # Вычисляем RMSE
    rmse = np.sqrt(mean_squared_error(test['rating'], test['predicted_rating']))
    return rmse

# 4. Рекомендательная система на основе кластеризации и расчета средней оценки кластера
def cluster_based_cf(train, test, n_clusters=40):
    # Создаем матрицу пользователь-фильм
    user_item_matrix = train.pivot(index='userId', columns='movieId', values='rating').fillna(0)
    
    # Применяем K-means кластеризацию
    kmeans = KMeans(n_clusters=n_clusters, random_state=42)
    user_clusters = kmeans.fit_predict(user_item_matrix)
    
    # Функция для предсказания рейтинга
    def predict_rating(user_id, item_id):
        if user_id not in user_item_matrix.index or item_id not in user_item_matrix.columns:
            return user_item_matrix.mean().mean()
        
        user_cluster = user_clusters[user_item_matrix.index.get_loc(user_id)]
        cluster_users = user_item_matrix.index[user_clusters == user_cluster]
        cluster_ratings = user_item_matrix.loc[cluster_users, item_id]
        
        if cluster_ratings.empty:
            return user_item_matrix.mean().mean()
        return cluster_ratings.mean()
    
    # Предсказываем рейтинги для тестового набора
    test['predicted_rating'] = test.apply(lambda row: predict_rating(row['userId'], row['movieId']), axis=1)
    
    # Вычисляем RMSE
    rmse = np.sqrt(mean_squared_error(test['rating'], test['predicted_rating']))
    return rmse

# 5. Рекомендательная система на основе кластеризации и учета z-оценки
def cluster_based_cf_z_score(train, test, n_clusters=40):
    # Создаем матрицу пользователь-фильм
    user_item_matrix = train.pivot(index='userId', columns='movieId', values='rating').fillna(0)
    
    # Нормализуем оценки с помощью z-score
    user_means = user_item_matrix.mean(axis=1)
    user_stds = user_item_matrix.std(axis=1)
    user_item_matrix_normalized = user_item_matrix.sub(user_means, axis=0).div(user_stds, axis=0)
    
    # Применяем K-means кластеризацию
    kmeans = KMeans(n_clusters=n_clusters, random_state=42)
    user_clusters = kmeans.fit_predict(user_item_matrix_normalized)
    
    # Функция для предсказания рейтинга
    def predict_rating(user_id, item_id):
        if user_id not in user_item_matrix.index or item_id not in user_item_matrix.columns:
            return user_item_matrix.mean().mean()
        
        user_cluster = user_clusters[user_item_matrix.index.get_loc(user_id)]
        cluster_users = user_item_matrix.index[user_clusters == user_cluster]
        cluster_ratings_normalized = user_item_matrix_normalized.loc[cluster_users, item_id]
        
        if cluster_ratings_normalized.empty:
            return user_means[user_id]
        
        predicted_z_score = cluster_ratings_normalized.mean()
        return user_means[user_id] + (predicted_z_score * user_stds[user_id])
    
    # Предсказываем рейтинги для тестового набора
    test['predicted_rating'] = test.apply(lambda row: predict_rating(row['userId'], row['movieId']), axis=1)
    
    # Вычисляем RMSE
    rmse = np.sqrt(mean_squared_error(test['rating'], test['predicted_rating']))
    return rmse

# 6. Рекомендательная система на основе голосования
def voting_based_cf(train, test, n_neighbors=40):
    # Создаем матрицу пользователь-фильм
    user_item_matrix = train.pivot(index='userId', columns='movieId', values='rating').fillna(0)
    
    # Вычисляем косинусное сходство между пользователями
    user_similarity = cosine_similarity(user_item_matrix)
    
    # Функция для предсказания рейтинга
    def predict_rating(user_id, item_id):
        if user_id not in user_item_matrix.index or item_id not in user_item_matrix.columns:
            return user_item_matrix.mean().mean()
        
        user_ratings = user_item_matrix.loc[user_id]
        similar_users = user_similarity[user_item_matrix.index.get_loc(user_id)]
        
        # Находим топ-N похожих пользователей
        top_similar_users = np.argsort(similar_users)[-n_neighbors-1:-1]
        
        # Собираем голоса
        votes = user_item_matrix.iloc[top_similar_users][item_id].values
        votes = votes[votes > 0]
        
        if len(votes) == 0:
            return user_ratings.mean()
        
        # Возвращаем наиболее популярную оценку
        return np.bincount(votes.astype(int)).argmax()
    
    # Предсказываем рейтинги для тестового набора
    test['predicted_rating'] = test.apply(lambda row: predict_rating(row['userId'], row['movieId']), axis=1)
    
    # Вычисляем RMSE
    rmse = np.sqrt(mean_squared_error(test['rating'], test['predicted_rating']))
    return rmse

# Оформление результа
def create_results_dataframe(results):
    """
    Создает pandas DataFrame из словаря результатов.
    
    :param results: Словарь с названиями методов в качестве ключей и RMSE в качестве значений
    :return: pandas DataFrame с результатами
    """
    df = pd.DataFrame(list(results.items()), columns=['Метод', 'RMSE'])
    df.set_index('Метод', inplace=True)
    return df

# Основная функция
def main():
    # Загрузка и разделение данных
    train, test = load_and_split_data('data/ml-latest-small/ratings.csv')
    
    results = {}

    # User-based CF
    results['user_based'] = user_based_cf(train, test)

    # User-based CF with Mean
    results['user_based_mean'] = user_based_cf(train, test, use_mean_adjustment=True)
    
    # Item-based CF
    results['item_based'] = item_based_cf(train, test)

    # Item-based CF with Mean
    results['item_based_mean'] = item_based_cf(train, test, use_mean_adjustment=True)
    
    # Cluster-based CF
    results['cluster_based'] = cluster_based_cf(train, test)
    
    # Cluster-based CF with z-score
    results['cluster_based_z_score'] = cluster_based_cf_z_score(train, test)
    
    # Voting-based CF
    results['voting_based'] = voting_based_cf(train, test)
    
    # Создаем DataFrame с результатами
    results_df = create_results_dataframe(results)
    
    # Выводим результаты
    print(results_df.sort_values('RMSE'))

if __name__ == "__main__":
    main()

ValueError: setting an array element with a sequence.

                     RMSE
Метод                    
SVD-based CF     0.880165
Voting-based CF  0.881114
User-based CF    0.907214
Item-based CF    0.910994
KNN-Z CF         0.916671
NMF-based CF     0.970052
voting_based           1.530395

**Итог 1:**
Голосование показало себя лучше всего.

*Нужно посмотреть данные и понять почему `user_based` лучше, чем `item_based`*

In [9]:
import pandas as pd
import numpy as np
from surprise import Dataset, Reader, SVD, KNNBasic, KNNWithMeans, KNNWithZScore, NMF
from surprise.model_selection import train_test_split
from surprise.accuracy import rmse
from collections import defaultdict

# 1. Загрузка и разделение данных
def load_and_split_data(file_path, test_size=0.3, random_state=42):
    
    # Загрузка данных
    df = pd.read_csv(file_path)
    reader = Reader(rating_scale=(0.5, 5))
    data = Dataset.load_from_df(df[['userId', 'movieId', 'rating']], reader)
    
    # Разделение данных
    trainset, testset = train_test_split(data, test_size=test_size, random_state=random_state)
    return trainset, testset

# 2. User-based коллаборативная фильтрация
def user_based_cf(trainset, testset):
    sim_options = {'name': 'cosine', 'user_based': True}
    algo = KNNWithMeans(sim_options=sim_options, verbose=False)
    algo.fit(trainset) 
    predictions = algo.test(testset)
    return rmse(predictions)

# 3. Item-based коллаборативная фильтрация
def item_based_cf(trainset, testset):
    sim_options = {'name': 'cosine', 'user_based': False}
    algo = KNNWithMeans(sim_options=sim_options, verbose=False)
    algo.fit(trainset)
    predictions = algo.test(testset)
    return rmse(predictions)

# 4. Рекомендательная система на основе SVD (аналог кластеризации)
def svd_based_cf(trainset, testset):
    algo = SVD(n_factors=100, random_state=42)
    algo.fit(trainset)
    predictions = algo.test(testset)
    return rmse(predictions)

# 5.1 Рекомендательная система на основе NMF (аналог кластеризации с учетом z-оценки)
def nmf_based_cf(trainset, testset):
    algo = NMF(n_factors=50, random_state=42)
    algo.fit(trainset)
    predictions = algo.test(testset)
    return rmse(predictions)

# 5.2 Рекомендательная система на основе кластеризации с учетом z-оценки
def knnz_based_cf(trainset, testset):
    sim_options = {'name': 'cosine', 'user_based': False}
    algo = KNNWithZScore(sim_options=sim_options, verbose=False)
    algo.fit(trainset)
    predictions = algo.test(testset)
    return rmse(predictions)

# 6. Рекомендательная система на основе голосования (усредненное предсказание)
def voting_based_cf(trainset, testset):
    # Используем несколько алгоритмов для голосования
    algos = [
        KNNWithMeans(sim_options={'name': 'cosine', 'user_based': True}, verbose=False),
        KNNWithMeans(sim_options={'name': 'cosine', 'user_based': False}, verbose=False),
        SVD(n_factors=100, random_state=42),
        NMF(n_factors=50, random_state=42)
    ]
    
    # Обучаем все алгоритмы
    for algo in algos:
        algo.fit(trainset)
    
    # Предсказываем рейтинги и усредняем их
    predictions = defaultdict(list)
    for algo in algos:
        for prediction in algo.test(testset):
            uid, iid, _, est, _ = prediction
            predictions[(uid, iid)].append(est)
    
    # Усредняем предсказания
    final_predictions = []
    for (uid, iid), preds in predictions.items():
        true_r = next((true_r for (u, i, true_r) in testset if u == uid and i == iid), None)
        if true_r is not None:
            final_predictions.append((uid, iid, true_r, np.mean(preds), {'n_votes': len(preds)}))
    
    return rmse(final_predictions)

# Оформление результа
def create_results_dataframe(results):
    """
    Создает pandas DataFrame из словаря результатов.
    
    :param results: Словарь с названиями методов в качестве ключей и RMSE в качестве значений
    :return: pandas DataFrame с результатами
    """
    df = pd.DataFrame(list(results.items()), columns=['Метод', 'RMSE'])
    df.set_index('Метод', inplace=True)
    return df

# Основная функция
def main():
    # Загрузка и разделение данных
    trainset, testset = load_and_split_data('data/ml-latest-small/ratings.csv')
    
    # Словарь для хранения результатов
    results = {}
    
    # User-based CF
    results['User-based CF'] = user_based_cf(trainset, testset)
    
    # Item-based CF
    results['Item-based CF'] = item_based_cf(trainset, testset)
    
    # SVD-based CF (аналог кластеризации)
    results['SVD-based CF'] = svd_based_cf(trainset, testset)
    
    # NMF-based CF (аналог кластеризации с учетом z-оценки)
    results['NMF-based CF'] = nmf_based_cf(trainset, testset)

    #KNN-Z CF
    results['KNN-Z CF'] = knnz_based_cf(trainset, testset)
    
    # Voting-based CF
    results['Voting-based CF'] = voting_based_cf(trainset, testset)
    
    # Создаем DataFrame с результатами
    results_df = create_results_dataframe(results)
    
    # Выводим результаты
    print(results_df.sort_values('RMSE'))
    
    # results_df.to_csv('recommender_systems_results.csv')

if __name__ == "__main__":
    main()

RMSE: 0.9072
RMSE: 0.9110
RMSE: 0.8802
RMSE: 0.9701
RMSE: 0.9167
RMSE: 0.8811
                     RMSE
Метод                    
SVD-based CF     0.880165
Voting-based CF  0.881114
User-based CF    0.907214
Item-based CF    0.910994
KNN-Z CF         0.916671
NMF-based CF     0.970052
